In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [7]:
train = np.genfromtxt('../input/xxx/train.csv',delimiter=",")
test = np.genfromtxt("../input/xxx/test.csv",delimiter=",")

In [ ]:
class NeuralNet(torch.nn.Module):
    def __init__(self, lrate,loss_fn,in_size,out_size,device,penalty=0):
        """
        Initialize the layers of your neural network

        @param lrate: The learning rate for the model.
        @param loss_fn: A loss function defined in the following way:
            @param yhat - an (N,out_size) tensor
            @param y - an (N,) tensor
            @return l(x,y) an () tensor that is the mean loss
        @param in_size: Dimension of input
        @param out_size: Dimension of output
        """
        super(NeuralNet, self).__init__()
        self.loss_fn = loss_fn
        self.lrate = lrate
        self.loss_fn = loss_fn
        self.device=device
        self.conv = torch.nn.Sequential(
            
            )
        self.model = torch.nn.Sequential(
            )
        self.optim = torch.optim.Adam([x for x in self.model.parameters()] + [x for x in self.conv.parameters()],lr=self.lrate,weight_decay=penalty)

    def forward(self, x):
        """ A forward pass of your neural net (evaluates f(x)).

        @param x: an (N, in_size) torch tensor

        @return y: an (N, out_size) torch tensor of output from the network
        """

    def step(self, x,y):
        """
        Performs one gradient step through a batch of data x with labels y
        @param x: an (N, in_size) torch tensor
        @param y: an (N,) torch tensor
        @return L: total empirical risk (mean of losses) at this time step as a float
        """
        self.optim.zero_grad()
        output = self.forward(x)
        loss = self.loss_fn(output,y)

        loss.backward()
        self.optim.step()
        return loss

In [ ]:
def compute_accuracies(predicted_labels,dev_set,dev_labels):
    yhats = predicted_labels
    if len(yhats) != len(dev_labels):
        print("Lengths of predicted labels don't match length of actual labels", len(yhats),len(dev_labels))
        return 0.,0.,0.,0.
    accuracy = np.mean(yhats == dev_labels)
    tp = np.sum([yhats[i] == dev_labels[i] and yhats[i] == 1 for i in range(len(dev_labels))])
    precision = tp / np.sum([yhats[i]==1 for i in range(len(dev_labels))])
    recall = tp / (np.sum([yhats[i] != dev_labels[i] and yhats[i] == 0 for i in range(len(dev_labels))]) + tp)
    f1 = 2 * (precision * recall) / (precision + recall)
    
    return accuracy,f1,precision,recall

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

params = {'batch_size': 25,
          'shuffle': True,
          'num_workers' : 8}

train_data = torch.tensor(train_d, dtype=torch.float)
train_labels = torch.tensor(train_l, dtype=torch.long)
test_data = torch.tensor(test_d,dtype=torch.float)

trainset = torch.utils.data.TensorDataset(train_data, train_labels)
trainloader=torch.utils.data.DataLoader(trainset, **params)
net = NeuralNet(1e-4,torch.nn.CrossEntropyLoss(),train_data.shape[1],10,device,penalty=.01)
yhats = np.zeros((test_data.shape[0],))

for epoch in range(5):
    for inputs,lbls in trainloader:
        inputs, lbls = inputs.float().to(device), lbls.long().to(device)
        cur_loss = net.step(inputs,lbls)
for i in range(test_data.shape[0]): #dev_set.shape[0]
    inputs = test_data[i,:].to(device)
    out = net.forward(inputs)
    max_val, max_idx = out[0].max(0)
    yhats[i] = max_idx.item()

accuracy,f1,precision,recall = compute_accuracies(yhats,test_data,test_l)
print("accuracy: {}".format(accuracy))
print("f1: {}".format(f1))
print("precision: {}".format(precision))
print("recall: {}".format(recall))